In [1]:
## Parse PDB files into a dictionary
# use parse_multiple_chains.py; parse_multiple_chains.sh

In [ ]:
import matplotlib.pyplot as plt

In [9]:
from dateutil import parser
import numpy as np
import os, time, gzip, json
import glob 
import argparse

# argparser = argparse.ArgumentParser(formatter_class=argparse.ArgumentDefaultsHelpFormatter)

# argparser.add_argument("--pdb_folder", type=str, help="Path to a folder with pdb files, e.g. /home/my_pdbs/")
# argparser.add_argument("--out_path", type=str, help="Path where to save .jsonl dictionary of parsed pdbs")

# args = argparser.parse_args()

# folder_with_pdbs_path = args.pdb_folder
# save_path = args.out_path

#MODIFY THIS PATH TO YOUR FOLDER WITH PDBS
folder_with_pdbs_path = 'combined/'
#MODIFY OUTPUT PATH
save_path = 'pdbs_test.jsonl'

alpha_1 = list("ARNDCQEGHILKMFPSTWYV-")
states = len(alpha_1)
alpha_3 = ['ALA','ARG','ASN','ASP','CYS','GLN','GLU','GLY','HIS','ILE',
           'LEU','LYS','MET','PHE','PRO','SER','THR','TRP','TYR','VAL','GAP']

aa_1_N = {a:n for n,a in enumerate(alpha_1)}
aa_3_N = {a:n for n,a in enumerate(alpha_3)}
aa_N_1 = {n:a for n,a in enumerate(alpha_1)}
aa_1_3 = {a:b for a,b in zip(alpha_1,alpha_3)}
aa_3_1 = {b:a for a,b in zip(alpha_1,alpha_3)}

def AA_to_N(x):
  # ["ARND"] -> [[0,1,2,3]]
  x = np.array(x);
  if x.ndim == 0: x = x[None]
  return [[aa_1_N.get(a, states-1) for a in y] for y in x]

def N_to_AA(x):
  # [[0,1,2,3]] -> ["ARND"]
  x = np.array(x);
  if x.ndim == 1: x = x[None]
  return ["".join([aa_N_1.get(a,"-") for a in y]) for y in x]


def parse_PDB_biounits(x, atoms=['N','CA','C'], chain=None):
  '''
  input:  x = PDB filename
          atoms = atoms to extract (optional)
  output: (length, atoms, coords=(x,y,z)), sequence
  '''
  xyz,seq,min_resn,max_resn = {},{},1e6,-1e6
  for line in open(x,"rb"):
    line = line.decode("utf-8","ignore").rstrip()

    if line[:6] == "HETATM" and line[17:17+3] == "MSE":
      line = line.replace("HETATM","ATOM  ")
      line = line.replace("MSE","MET")

    if line[:4] == "ATOM":
      ch = line[21:22]
      if ch == chain or chain is None:
        atom = line[12:12+4].strip()
        resi = line[17:17+3]
        resn = line[22:22+5].strip()
        x,y,z = [float(line[i:(i+8)]) for i in [30,38,46]]

        if resn[-1].isalpha(): 
            resa,resn = resn[-1],int(resn[:-1])-1
        else: 
            resa,resn = "",int(resn)-1
#         resn = int(resn)
        if resn < min_resn: 
            min_resn = resn
        if resn > max_resn: 
            max_resn = resn
        if resn not in xyz: 
            xyz[resn] = {}
        if resa not in xyz[resn]: 
            xyz[resn][resa] = {}
        if resn not in seq: 
            seq[resn] = {}
        if resa not in seq[resn]: 
            seq[resn][resa] = resi

        if atom not in xyz[resn][resa]:
          xyz[resn][resa][atom] = np.array([x,y,z])

  # convert to numpy arrays, fill in missing values
  seq_,xyz_ = [],[]
  try:
      for resn in range(min_resn,max_resn+1):
        if resn in seq:
          for k in sorted(seq[resn]): seq_.append(aa_3_N.get(seq[resn][k],20))
        else: seq_.append(20)
        if resn in xyz:
          for k in sorted(xyz[resn]):
            for atom in atoms:
              if atom in xyz[resn][k]: xyz_.append(xyz[resn][k][atom])
              else: xyz_.append(np.full(3,np.nan))
        else:
          for atom in atoms: xyz_.append(np.full(3,np.nan))
      return np.array(xyz_).reshape(-1,len(atoms),3), N_to_AA(np.array(seq_))
  except TypeError:
      return 'no_chain', 'no_chain'



pdb_dict_list = []
c = 0

if folder_with_pdbs_path[-1]!='/':
    folder_with_pdbs_path = folder_with_pdbs_path+'/'


init_alphabet = ['A', 'B', 'C', 'D', 'E', 'F', 'G','H', 'I', 'J','K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T','U', 'V','W','X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g','h', 'i', 'j','k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't','u', 'v','w','x', 'y', 'z']
extra_alphabet = [str(item) for item in list(np.arange(300))]
chain_alphabet = init_alphabet + extra_alphabet

biounit_names = glob.glob(folder_with_pdbs_path+'*asym.pdb')
count = 0
for biounit in biounit_names:
    count+=1
    print (count)
    my_dict = {}
    s = 0
    concat_seq = ''
    concat_N = []
    concat_CA = []
    concat_C = []
    concat_O = []
    concat_mask = []
    coords_dict = {}
    for letter in chain_alphabet:
        xyz, seq = parse_PDB_biounits(biounit, atoms=['N','CA','C','O'], chain=letter)
        if type(xyz) != str:
            concat_seq += seq[0]
            my_dict['seq_chain_'+letter]=seq[0]
            coords_dict_chain = {}
            coords_dict_chain['N_chain_'+letter]=xyz[:,0,:].tolist()
            coords_dict_chain['CA_chain_'+letter]=xyz[:,1,:].tolist()
            coords_dict_chain['C_chain_'+letter]=xyz[:,2,:].tolist()
            coords_dict_chain['O_chain_'+letter]=xyz[:,3,:].tolist()
            my_dict['coords_chain_'+letter]=coords_dict_chain
            s += 1
    fi = biounit.rfind("/")
    my_dict['name']=biounit[(fi+1):-4]
    my_dict['num_of_chains'] = s
    my_dict['seq'] = concat_seq
    if s < len(chain_alphabet):
        pdb_dict_list.append(my_dict)
        c+=1
        
        
with open(save_path, 'w') as f:
    for entry in pdb_dict_list:
        f.write(json.dumps(entry) + '\n')
        
print(f'Successfully finished: {len(biounit_names)} pdbs')


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [12]:
#MAKE masked/visible chains dictionary
#Masked means the chains that need to be designed, e.g. binder
#Visible means the chains that are fixed, they won't be modelled but only used as a context, e.g. target
# use make_masked_visible_chain_dict.py; make_masked_visible_chain_dict.sh

In [12]:
import glob
import random

import json

def no_of_chains(filename):
                # /home/apillai1/second_rings_set/one_component_dihedral/worms_file3/
    infile = open("combined/"+filename+".pdb","r")
    chain = []
    for line in infile:
      if "ATOM" in line:
        k = line.split()
        if len(k[4])==1:
          chain_id = k[4]
        elif len(k[4])>1:
           chain_id = k[4][0]
        if not chain_id in chain:
          chain.append(chain_id)
    print (chain)
    infile.close()
    return chain
    


#MODIFY THIS PATH - it is a path to the parsed pdb files
            
with open('pdbs_test.jsonl', 'r') as json_file:
    json_list = list(json_file)
count = 0
my_dict = {}
for json_str in json_list:
    result = json.loads(json_str)
    all_chain_list = [item[-1:] for item in list(result) if item[:9]=='seq_chain'] #['A','B', 'C',...]
    filename = result['name']
    k = no_of_chains(filename)
   # if k ==1:
    masked_chain_list = ['A'] 

    visible_chain_list = ['B'] #allow to use chain B as a context
    my_dict[result['name']]= (masked_chain_list, visible_chain_list)
    count+=1
    print ("Hello",count,result['name'])
#MODIFY THIS PATH FOR THE OUTPUT DICTIONARY
count = 0
with open('pdbs_masked.jsonl', 'w') as f:
    count+=1
#    print ("Hello",count, results['name'])
    f.write(json.dumps(my_dict) + '\n')


print('Finished')
# Output looks like this:
# {"5TTA": [["A"], ["B"]], "3LIS": [["A"], ["B"]]}

['A', 'B']
Hello 1 C3_worms_mbb0002__0267_81-236_JHB7Yr2.pdb_ex179_en126_LHD289_DHR82_BA_A_0003_0012.pdb_ex380_en24_JHB7Yr2.pdb_ex380_en24_ABAA_asym
['A', 'B']
Hello 2 C4_worms_mbb0002__0414_75-191_JHB7Yr2.pdb_ex168_en90_LHD321_DHR54_BA_A_0001_0001.pdb_ex338_en52_JHB7Yr2.pdb_ex338_en52_AFAB_asym
['A', 'B']
Hello 3 C2_worms_mbb0001__0035_121-301_FF74Y.pdb_ex198_en146_LHD289_DHR53_BA_A_0005_0013.pdb_ex378_en18_FF74Y.pdb_ex378_en18_ABAB_asym
['A', 'B']
Hello 4 C2_worms_mbb0002__0271_108-286_JHB7Yr2.pdb_ex185_en143_LHD321_DHR53_BA_A_0015_0003.pdb_ex340_en7_JHB7Yr2.pdb_ex340_en7_ABBB_asym
['A', 'B']
Hello 5 C5_worms_mbb0001__0287_82-225_FF74Y.pdb_ex181_en93_LHD289_DHR54_BA_A_0003_0007.pdb_ex360_en38_FF74Y.pdb_ex360_en38_ABAB_asym
['A', 'B']
Hello 6 C5_worms_mbb0002__0090_127-269_JHB7Yr2.pdb_ex175_en90_LHD321_DHR62_BA_A_0017_0003.pdb_ex359_en33_JHB7Yr2.pdb_ex359_en33_ABAB_asym
['A', 'B']
Hello 7 C4_worms_mbb0002__0295_102-242_JHB7Yr2.pdb_ex169_en135_LHD317_DHR57_BA_A_0003_0003.pdb_ex369_en29